In [ ]:
import os
import numpy as np
import torch
import tqdm
import yaml
from attrdict import AttrDict
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import defaultdict

from modules.mono.depth_net_pl import depth_net_pl
from modules.semantic.semantic_net_pl import semantic_net_pl
from modules.semantic.semantic_net_pl_maskformer_small import semantic_net_pl_maskformer_smallpl
from modules.semantic.semantic_mapper import ObjectType
from data_utils.data_utils import GridSeqDataset
from data_utils.prob_vol_data_utils import ProbVolDataset
from utils.localization_utils import (
    get_ray_from_depth, get_ray_from_semantics_v2, localize, finalize_localization
)
from utils.data_loader_helper import load_scene_datad
from utils.visualization_utils import plot_dict_relationship
from scipy.ndimage import gaussian_filter1d
import seaborn as sns
import os
import pickle

# Hard-coded configuration
config = AttrDict({
    'dataset_dir': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/structured3d_perspective_full',
    'desdf_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/desdf',
    'log_dir_depth': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/modules/Final_wights/depth/final_depth_model_checkpoint.ckpt',
    'log_dir_semantic': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/modules/Final_wights/semantic/final_semantic_model_checkpoint.ckpt',
    'split_file': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/structured3d_perspective_full/split.yaml',
    'prob_vol_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/data/test_data_set_full/prob_vols',
    
    'results_dir': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/full/GT',
    'combined_prob_vols_net_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/logs/combined/expected_pose_pred/combined_prob_vols_net_type-expected_pose_pred_net_size-6k_dataset_size-medium_epochs-99/combined_net-epoch=58-loss-valid=7.16.ckpt',
    'combined_net_size': '6k',
    
    'L': 0,
    'D': 128,
    'd_min': 0.1,
    'd_max': 15.0,
    'd_hyp': -0.2,
    'F_W': 0.59587643422,
    'V': 7,
    'num_classes': 4,
    'prediction_type': 'combined',
    'use_ground_truth_depth': False,
    'use_ground_truth_semantic': False,
    'use_saved_prob_vol': False,
    'num_of_scenes': 1,
    'pad_to_max': True,
    'max_h': 1760,
    'max_w': 1760,
    'weight_combinations': [
        [1.0, 0.0],
        # [0, 1.0],
        [0.5, 0.5]
    ],    
    'use_semantic_mask2foremr': True,
    'semantic_mask2former_path': '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/logs/mask2Former_semantics_with_tiny_small_version_no_augmentaion/semantic/semantic_net-epoch=19-loss-valid=0.34.ckpt'
})

def save_data(data, path):
    """Saves data to a specified path using pickle."""
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load_data(path):
    """Loads data from a specified path using pickle."""
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def pad_to_max(prob_vol, max_H, max_W):
    return F.pad(prob_vol, (0, max_W - prob_vol.shape[1], 0, max_H - prob_vol.shape[0]))


def plot_acc_vs_count(count_dict_for_all_weights, count_name, save_path, data_type):
    """
    Plots accuracy vs number of objects for different weight combinations with smoothing and range shading.
    :param count_dict_for_all_weights: Dictionary containing accuracy values for different weights.
    :param count_name: Type of object (e.g., windows, doors, doors_and_walls).
    :param save_path: Path to save the plot.
    :param data_type: Data type ('pred' or 'gt').
    """
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 7))
    
    for weight_key in count_dict_for_all_weights.keys():
        sorted_counts = sorted(count_dict_for_all_weights[weight_key].keys())
        mean_acc = [np.mean(count_dict_for_all_weights[weight_key][c]) for c in sorted_counts]
        std_acc = [np.std(count_dict_for_all_weights[weight_key][c]) for c in sorted_counts]
        
        smoothed_mean = gaussian_filter1d(mean_acc, sigma=1)
        
        # Upper and lower bounds
        lower_bound = np.maximum(np.array(mean_acc) - np.array(std_acc), 0)  # Ensure lower bound is >= 0
        upper_bound = np.array(mean_acc) + np.array(std_acc)
        # Plot the smoothed mean
        sns.lineplot(x=sorted_counts, y=smoothed_mean, label=f'{weight_key}', linewidth=2)
        sns.lineplot(x=sorted_counts, y=mean_acc, linewidth=1, alpha=0.4)
        
        # Add shaded range for bounds (unsmoothed)
        # plt.fill_between(sorted_counts, lower_bound, upper_bound, alpha=0.05)

    plt.xlabel(f'Number of {count_name} Rays')
    plt.ylabel('Mean Accuracy (m)')
    # plt.title(f'Mean Accuracy vs Number of {count_name} Rays ({data_type.upper()})')
    plt.legend(title="Weight Combination")
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, f'{data_type}_acc_vs_{count_name}.png'))
    plt.close()



def evaluate_combined_model(
    depth_net, semantic_net, desdfs, semantics, test_set, gt_poses, maps,
    device, results_type_dir, valid_scene_names, config
):
    # Define file paths to save intermediate results
    save_path = '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/final_results/mask2former/visualtizations/semantic_vs_acc'
    
    os.makedirs(save_path, exist_ok=True)
    counts_file = os.path.join(save_path, 'count_dict_for_all_weights.pkl')
    
    if os.path.exists(counts_file):
        # Load existing data if it exists
        print(f"Loading existing count data from {counts_file}")
        data = load_data(counts_file)
        windows_acc = data['windows_acc']
        doors_acc = data['doors_acc']
        doors_and_windows_acc = data['doors_and_windows_acc']
        gt_windows_acc = data['gt_windows_acc']
        gt_doors_acc = data['gt_doors_acc']
        gt_doors_and_windows_acc = data['gt_doors_and_windows_acc']
    else:
        print(f"No existing data found at {counts_file}, running calculations.")
        wc = config.weight_combinations
        acc_records = {f"{dw}_{sw}": [] for dw, sw in wc}
        acc_orn_records = {f"{dw}_{sw}": [] for dw, sw in wc}
        all_weight_comb_dict = {f"{dw}_{sw}": {} for dw, sw in wc}
        windows_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}
        doors_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}
        doors_and_windows_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}

        # Dictionaries for GT data
        gt_windows_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}
        gt_doors_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}
        gt_doors_and_windows_acc = {f"{dw}_{sw}": defaultdict(list) for dw, sw in wc}

        for data_idx in tqdm.tqdm(range(len(test_set))):
            data = test_set[data_idx]
            scene_idx = np.sum(data_idx >= np.array(test_set.scene_start_idx)) - 1
            scene = test_set.scene_names[scene_idx]
            if 'floor' not in scene:
                scene = f"scene_{int(scene.split('_')[1])}"
            if scene not in valid_scene_names:
                continue

            idx_within_scene = data_idx - test_set.scene_start_idx[scene_idx]
            desdf, semantic = desdfs[scene], semantics[scene]
            ref_pose_map = gt_poses[scene][idx_within_scene * (config.L + 1) + config.L]

            pred_depths = depth_net.encoder(torch.tensor(data["ref_img"], device=device).unsqueeze(0), None)[0].squeeze(0).cpu().detach().numpy()
            pred_rays_depth = get_ray_from_depth(pred_depths, V=config.V, F_W=config.F_W)
            
            gt_pred_depths = data["ref_depth"]
            gt_rays_depth = get_ray_from_depth(gt_pred_depths, V=config.V, F_W=config.F_W)

            ref_img_t = torch.tensor(data["ref_img"], device=device).unsqueeze(0)
            _, _, prob = semantic_net.encoder(ref_img_t, None)
            sampled_indices_np = torch.multinomial(prob.squeeze(0), 1, replacement=True).squeeze(1).cpu().detach().numpy()
            pred_rays_semantic = get_ray_from_semantics_v2(sampled_indices_np)
            
            gt_sampled_indices_np = data["ref_semantics"]
            gt_rays_semantic = get_ray_from_semantics_v2(gt_sampled_indices_np)

            prob_vol_pred_depth = localize(torch.tensor(desdf["desdf"]), torch.tensor(pred_rays_depth), return_np=False)[0]
            prob_vol_pred_semantic = localize(torch.tensor(semantic["desdf"]), torch.tensor(pred_rays_semantic), return_np=False,localize_type="semantic")[0]
            
            gt_prob_vol_pred_depth = localize(torch.tensor(desdf["desdf"]), torch.tensor(gt_rays_depth), return_np=False)[0]
            gt_prob_vol_pred_semantic = localize(torch.tensor(semantic["desdf"]), torch.tensor(gt_rays_semantic), return_np=False,localize_type="semantic")[0]

            # Separate GT and semantic counts
            num_windows_pred = np.sum(sampled_indices_np == ObjectType.WINDOW.value)
            num_doors_pred = np.sum(sampled_indices_np == ObjectType.DOOR.value)
            num_doors_and_windows_pred = np.sum((sampled_indices_np == ObjectType.DOOR.value) | (sampled_indices_np == ObjectType.WINDOW.value))

            num_windows_gt = np.sum(gt_sampled_indices_np == ObjectType.WINDOW.value)
            num_doors_gt = np.sum(gt_sampled_indices_np == ObjectType.DOOR.value)
            num_doors_and_windows_gt = np.sum((gt_sampled_indices_np == ObjectType.DOOR.value) | (gt_sampled_indices_np == ObjectType.WINDOW.value))

            for dw, sw in wc:
                wkey = f"{dw}_{sw}"
                min_shape = [min(d, s) for d, s in zip(prob_vol_pred_depth.shape, prob_vol_pred_semantic.shape)]
                d_sliced = prob_vol_pred_depth[tuple(slice(0, m) for m in min_shape)]
                s_sliced = prob_vol_pred_semantic[tuple(slice(0, m) for m in min_shape)]
                combined_prob = dw * d_sliced + sw * s_sliced
                _, pdp, _, pose_pred = finalize_localization(combined_prob)
                pose_pred = torch.tensor(pose_pred, device=device, dtype=torch.float32)
                pose_pred[:2] = pose_pred[:2] / 10
                acc = torch.norm(pose_pred[:2] - torch.tensor(ref_pose_map[:2], device=device), 2).item()
                acc_orn = ((pose_pred[2] - ref_pose_map[2]) % (2 * np.pi))
                acc_orn = min(acc_orn, 2 * np.pi - acc_orn) / np.pi * 180
                key = pdp.max().item()
                all_weight_comb_dict[wkey].setdefault(key, []).append(acc)
                acc_records[wkey].append(acc)
                acc_orn_records[wkey].append(acc_orn)
                windows_acc[wkey][num_windows_pred].append(acc)
                doors_acc[wkey][num_doors_pred].append(acc)
                doors_and_windows_acc[wkey][num_doors_and_windows_pred].append(acc)
                
                min_shape = [min(d, s) for d, s in zip(gt_prob_vol_pred_depth.shape, gt_prob_vol_pred_semantic.shape)]
                d_gt_sliced = gt_prob_vol_pred_depth[tuple(slice(0, m) for m in min_shape)]
                s_gt_sliced = gt_prob_vol_pred_semantic[tuple(slice(0, m) for m in min_shape)]
                gt_combined_prob = dw * d_gt_sliced + sw * s_gt_sliced
                _, gt_pdp, _, gt_pose_pred = finalize_localization(gt_combined_prob)
                gt_pose_pred = torch.tensor(gt_pose_pred, device=device, dtype=torch.float32)
                gt_pose_pred[:2] = gt_pose_pred[:2] / 10
                gt_acc = torch.norm(gt_pose_pred[:2] - torch.tensor(ref_pose_map[:2], device=device), 2).item()
                gt_windows_acc[wkey][num_windows_gt].append(gt_acc)
                gt_doors_acc[wkey][num_doors_gt].append(gt_acc)
                gt_doors_and_windows_acc[wkey][num_doors_and_windows_gt].append(gt_acc)
                
        # Save the computed results
        data_to_save = {
            'windows_acc': windows_acc,
            'doors_acc': doors_acc,
            'doors_and_windows_acc': doors_and_windows_acc,
            'gt_windows_acc': gt_windows_acc,
            'gt_doors_acc': gt_doors_acc,
            'gt_doors_and_windows_acc': gt_doors_and_windows_acc
        }
        save_data(data_to_save, counts_file)


    save_path = '/datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/final_results/visualtizations/semantic_vs_acc'
    os.makedirs(save_path, exist_ok=True)

    plot_acc_vs_count(windows_acc, 'windows', save_path, 'pred')
    plot_acc_vs_count(doors_acc, 'doors', save_path, 'pred')
    plot_acc_vs_count(doors_and_windows_acc, 'doors_and_windows', save_path, 'pred')
    
    plot_acc_vs_count(gt_windows_acc, 'windows', save_path, 'gt')
    plot_acc_vs_count(gt_doors_acc, 'doors', save_path, 'gt')
    plot_acc_vs_count(gt_doors_and_windows_acc, 'doors_and_windows', save_path, 'gt')



def evaluate_observation(prediction_type, config, device):
    with open(config.split_file, "r") as f:
        split = AttrDict(yaml.safe_load(f))
    scene_names = split.test[:config.num_of_scenes] if config.num_of_scenes>0 else split.test
    test_set = ProbVolDataset(config.dataset_dir, scene_names, L=config.L, prob_vol_path=config.prob_vol_path, acc_only=False) if config.use_saved_prob_vol \
               else GridSeqDataset(config.dataset_dir, scene_names, L=config.L)

    depth_net, semantic_net = None, None
    if not config.use_ground_truth_depth or not config.use_ground_truth_semantic:
        if prediction_type in ["depth", "combined"]:
            depth_net = depth_net_pl.load_from_checkpoint(config.log_dir_depth, d_min=config.d_min, d_max=config.d_max, d_hyp=config.d_hyp, D=config.D).to(device)
        if prediction_type in ["semantic", "combined"]:
            if config.use_semantic_mask2foremr:            
                semantic_net = semantic_net_pl_maskformer_smallpl.load_from_checkpoint(config.semantic_mask2former_path, num_classes=config.num_classes).to(device)
            else:
                semantic_net = semantic_net_pl.load_from_checkpoint(config.log_dir_semantic, num_classes=config.num_classes).to(device)

    results_type_dir = os.path.join(config.results_dir, prediction_type)
    if config.use_ground_truth_depth or config.use_ground_truth_semantic:
        results_type_dir = os.path.join(results_type_dir, "gt")
    os.makedirs(results_type_dir, exist_ok=True)

    desdfs, semantics, maps, gt_poses, valid_scene_names = load_scene_data(test_set, config.dataset_dir, config.desdf_path)
    evaluate_combined_model(depth_net, semantic_net, desdfs, semantics, test_set, gt_poses, maps, device, results_type_dir, valid_scene_names, config)

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if config.prediction_type == "all":
        for pt in ["depth", "semantic", "combined"]:
            evaluate_observation(pt, config, device)
    else:
        evaluate_observation(config.prediction_type, config, device)

if __name__ == "__main__":
    main()


/home/mascher/anaconda3/envs/yuval_fp/lib/python3.8/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/mascher/anaconda3/envs/yuval_

number of valid scenes for evaluation: 1 out of: 249 --> 0.4016064257028112%
number of valid scenes for evaluation: 2 out of: 249 --> 0.8032128514056224%
number of valid scenes for evaluation: 3 out of: 249 --> 1.2048192771084338%
number of valid scenes for evaluation: 4 out of: 249 --> 1.6064257028112447%
number of valid scenes for evaluation: 5 out of: 249 --> 2.0080321285140563%
number of valid scenes for evaluation: 6 out of: 249 --> 2.4096385542168677%
number of valid scenes for evaluation: 7 out of: 249 --> 2.8112449799196786%
number of valid scenes for evaluation: 8 out of: 249 --> 3.2128514056224895%
number of valid scenes for evaluation: 9 out of: 249 --> 3.614457831325301%


  4%|▍         | 10/249 [00:00<00:05, 41.71it/s]

number of valid scenes for evaluation: 10 out of: 249 --> 4.016064257028113%
number of valid scenes for evaluation: 11 out of: 249 --> 4.417670682730924%
number of valid scenes for evaluation: 12 out of: 249 --> 4.819277108433735%
number of valid scenes for evaluation: 13 out of: 249 --> 5.220883534136546%
number of valid scenes for evaluation: 14 out of: 249 --> 5.622489959839357%


  9%|▉         | 23/249 [00:00<00:05, 43.63it/s]

...

number of valid scenes for evaluation: 218 out of: 249 --> 87.55020080321285%
number of valid scenes for evaluation: 219 out of: 249 --> 87.95180722891565%
number of valid scenes for evaluation: 220 out of: 249 --> 88.35341365461848%
number of valid scenes for evaluation: 221 out of: 249 --> 88.75502008032129%
number of valid scenes for evaluation: 222 out of: 249 --> 89.1566265060241%
number of valid scenes for evaluation: 223 out of: 249 --> 89.5582329317269%
number of valid scenes for evaluation: 224 out of: 249 --> 89.95983935742971%
number of valid scenes for evaluation: 225 out of: 249 --> 90.36144578313254%
number of valid scenes for evaluation: 226 out of: 249 --> 90.76305220883533%
number of valid scenes for evaluation: 227 out of: 249 --> 91.16465863453816%
number of valid scenes for evaluation: 228 out of: 249 --> 91.56626506024097%


 95%|█████████▍| 236/249 [00:04<00:00, 55.38it/s]

number of valid scenes for evaluation: 229 out of: 249 --> 91.96787148594377%
number of valid scenes for evaluation: 230 out of: 249 --> 92.3694779116466%
number of valid scenes for evaluation: 231 out of: 249 --> 92.7710843373494%
number of valid scenes for evaluation: 232 out of: 249 --> 93.17269076305222%
number of valid scenes for evaluation: 233 out of: 249 --> 93.57429718875501%
number of valid scenes for evaluation: 234 out of: 249 --> 93.97590361445783%
number of valid scenes for evaluation: 235 out of: 249 --> 94.37751004016064%
number of valid scenes for evaluation: 236 out of: 249 --> 94.77911646586345%
number of valid scenes for evaluation: 237 out of: 249 --> 95.18072289156626%
number of valid scenes for evaluation: 238 out of: 249 --> 95.58232931726907%
number of valid scenes for evaluation: 239 out of: 249 --> 95.98393574297188%
number of valid scenes for evaluation: 240 out of: 249 --> 96.3855421686747%


100%|██████████| 249/249 [00:04<00:00, 56.72it/s]

number of valid scenes for evaluation: 241 out of: 249 --> 96.78714859437751%
number of valid scenes for evaluation: 242 out of: 249 --> 97.18875502008032%
number of valid scenes for evaluation: 243 out of: 249 --> 97.59036144578313%
number of valid scenes for evaluation: 244 out of: 249 --> 97.99196787148594%
number of valid scenes for evaluation: 245 out of: 249 --> 98.39357429718876%
number of valid scenes for evaluation: 246 out of: 249 --> 98.79518072289156%
number of valid scenes for evaluation: 247 out of: 249 --> 99.19678714859438%
number of valid scenes for evaluation: 248 out of: 249 --> 99.59839357429718%
number of valid scenes for evaluation: 249 out of: 249 --> 100.0%
Loading existing count data from /datadrive2/CRM.AI.Research/TeamFolders/Email/repo_yuval/FloorPlan/Semantic_Floor_plan_localization/results/final_results/visualtizations/semantic_vs_acc/count_dict_for_all_weights.pkl
